In [ ]:
# Code source: Sebastian Curi and Andreas Krause, based on Jaques Grobler (sklearn demos).
# License: BSD 3 clause

# We start importing some modules and running some magic commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# General math and plotting modules.
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import erfinv
from scipy import linalg

# Project files.
from utilities.util import gradient_descent
from utilities.classifiers import Logistic
from utilities.regularizers import L2Regularizer
from utilities.load_data import polynomial_data, linear_separable_data
from utilities import plot_helpers
from utilities.widgets import noise_widget, n_components_widget, min_prob_widget

# Widget and formatting modules
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed
from matplotlib import rcParams
import matplotlib as mpl 
from scipy.stats import multivariate_normal

# If in your browser the figures are not nicely vizualized, change the following line. 
rcParams['figure.figsize'] = (10, 5)
rcParams['font.size'] = 16

# Machine Learning library. 
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn import datasets

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def get_dataset(dataset, n_samples=200, noise=0.3):
    if dataset == 'linear':
        X, Y = linear_separable_data(n_samples, noise=noise, dim=2) 
        Y = (Y + 1) // 2
    elif dataset == '2-blobs':
        X, Y = datasets.make_classification(n_classes=2, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '3-blobs':
        X, Y = datasets.make_classification(n_classes=3, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '4-blobs':
        X, Y = datasets.make_classification(n_classes=4, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8) 
    elif dataset == 'high-dim':
        X, Y = datasets.make_classification(n_classes=3, n_features=10, n_informative=6, n_samples=n_samples,
                                            random_state=8)
        
    elif dataset == 'circles':
        X, Y = datasets.make_circles(noise=noise, n_samples=n_samples, factor=.5)
    elif dataset == 'moons':
        X, Y = datasets.make_moons(noise=noise, n_samples=n_samples)
    elif dataset == 'imbalanced':
        X, Y = linear_separable_data(n_samples, noise=noise, dim=2, num_negative=int(n_samples * 0.2))
        Y = (Y + 1) // 2
    elif dataset == 'correlated':
        X, Y = datasets.make_classification(n_classes=2, n_features=2, n_informative=1, n_redundant=1,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)  
    elif dataset == 'iris':
        X, Y = datasets.load_iris(return_X_y=True)
    elif dataset == 'mnist':
        X, Y = datasets.load_digits(return_X_y=True)
    elif dataset == 'wine':
        X, Y = datasets.load_wine(return_X_y=True)
    return X, Y

In [ ]:
def plot_ellipse(mean, covar, color, ax):
    v, w = linalg.eigh(covar)
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    u = w[0] / linalg.norm(w[0])

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan(u[1] / u[0])
    angle = 180. * angle / np.pi  # convert to degrees


    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle,  color=color)
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.5)
    ax.add_artist(ell)


def plot_model(model, X, Y, means, covariances):
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    pmap = plt.cm.cividis_r
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)
        
    # PREDICT
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]
    C = model.predict(xy)
    P = model.predict_proba(xy)
    H = -(P * model.predict_log_proba(xy)).sum(axis=1)
    
    P = P.max(axis=1)
    # Put the result into a color plot
    C = C.reshape(xx.shape)
    P = P.reshape(xx.shape)
    H = H.reshape(xx.shape)
    
    # PLOTS
    fig, axes = plt.subplots(1, 3)

    for ax in axes:
        ax.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.jet)
        
        if means is not None:
            for i in range(num_classes):
                plot_ellipse(means[i], covariances[i], cmap(norm(i)), ax)

        ax.set_xlim(xx.min()+h, xx.max()-h)
        ax.set_ylim(yy.min()+h, yy.max()-h)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_aspect('equal')

    axes[0].set_title('Classification Boundary')
    axes[0].contourf(xx, yy, C, cmap=cmap, alpha=0.5)
    
    axes[2].set_title('Prediction Probabilities')
    cf = axes[2].contourf(xx, yy, P, cmap=pmap, alpha=0.5, vmin=1. / num_classes, vmax=1)
    
    axes[1].set_title('Probabilistic Boundary')
    axes[1].contourf(xx, yy, P * C, cmap=cmap, alpha=0.5)
    plt.show()
    # Plot also the training point
    

# Generative Modelling (Classification)

In [ ]:
rcParams['figure.figsize'] = (20, 15)
rcParams['font.size'] = 16

def generative_modelling(dataset, model, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    num_classes = len(np.unique(Y))
    X = X[:, :2]
    if model == 'Naive Bayes':
        model = GaussianNB().fit(X, Y)
        mean, covariance = model.theta_, [np.diag(model.sigma_[i]) for i in range(len(model.classes_))]
        priors = model.class_prior_
    elif model == 'FisherLDA':
        model = LDA(store_covariance=True, priors=np.ones(num_classes) / num_classes).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_
    elif model == 'LDA':
        model = LDA(store_covariance=True).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_
    elif model == 'QDA':
        model = QDA(store_covariance=True, reg_param=1e-6).fit(X, Y)
        mean, covariance = model.means_, model.covariance_
        priors = model.priors_
    elif model == 'LogisticRegression':
        model = LogisticRegression().fit(X, Y)
        mean, covariance = None, None
        priors = None
        
    plot_model(model, X, Y, mean, covariance)
    if priors is not None:
        print(f"Class Priors: {priors}")
    
interact(generative_modelling, 
         dataset=['2-blobs', 'linear',  'correlated',  'imbalanced', '3-blobs', '4-blobs', 'circles', 'moons', 'iris'], 
         model=['Naive Bayes', 'FisherLDA', 'LDA', 'QDA', 'LogisticRegression'], noise=noise_widget);
        

# LDA as Dimensionality Reduction

In [ ]:
rcParams['figure.figsize'] = (20, 5)
rcParams['font.size'] = 16

dataset, noise = 'high-dim', 0.3
dataset = 'iris'

def dim_reduction(dataset, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)

    # X = X[:, :2]

    pca = PCA(n_components=2).fit(X)
    Xpca = pca.transform(X)
    
    if num_classes == 2:
        lda = LDA(n_components=2, store_covariance=True, solver='eigen').fit(X, Y)
        Xlda = np.dot(X, lda.scalings_)[:, :2]
        means = np.dot(lda.means_, lda.scalings_)[:, :2]
    else:
        lda = LDA(n_components=2, store_covariance=True).fit(X, Y)
        Xlda = np.dot(X - lda.xbar_, lda.scalings_)[:, :2]

        means = np.dot(lda.means_ - lda.xbar_, lda.scalings_)[:, :2]

    cov = lda.scalings_.T @ lda.covariance_ @ lda.scalings_ 
    cov = cov[:2, :2]

    fig, axes = plt.subplots(1, 3)

    axes[0].scatter(X[:, 0], X[:, 1], c=Y, cmap=cmap)
    axes[0].set_title('First 2 Dimensions')

    axes[1].scatter(Xpca[:, 0], Xpca[:, 1], c=Y, cmap=cmap)
    axes[1].set_title(f"PCA Explained Var {np.array2string(pca.explained_variance_ratio_, precision=2, separator=',')}")

    axes[2].scatter(Xlda[:, 0], Xlda[:, 1], c=Y, cmap=cmap)
    # axes[0].set_aspect('equal')
    for i in range(num_classes):
        plot_ellipse(means[i], cov, cmap(norm(i)), axes[2])
    axes[2].set_title(f"LDA Explained Var {np.array2string(lda.explained_variance_ratio_, precision=2, separator=',')}")
    
interact(
    dim_reduction, 
    dataset=['high-dim', 'iris',  'mnist', 'wine'], 
    noise=noise_widget
);


# Generative Modelling as Anomally Detection

In [ ]:
def get_anomalies(X, mean, covariance, priors, threshold):
    num_classes = len(covariance)
    P = np.zeros(X.shape[0])
    m = 0
    for i in range(num_classes):
        d = multivariate_normal(mean[i], covariance[i])
        d.logpdf(X)
        P += priors[i] * 2 * d.pdf(X)
        m += priors[i] * d.pdf(mean[i])
    return P < threshold

In [ ]:
rcParams['figure.figsize'] = (20, 10)
rcParams['font.size'] = 16

def anomaly_detection(dataset, threshold,  model_name, noise):
    np.random.seed(0)
    X, Y = get_dataset(dataset, noise=noise)
    X = X[:, :2]

    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)

    if model_name == 'Naive Bayes':
        model = GaussianNB().fit(X, Y)
        mean, covariance = model.theta_, [np.diag(model.sigma_[i]) for i in range(len(model.classes_))]
        priors = model.class_prior_
    elif model_name == 'FisherLDA':
        model = LDA(store_covariance=True, priors=np.ones(num_classes) / num_classes).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_
    elif model_name == 'LDA':
        model = LDA(store_covariance=True).fit(X, Y)
        mean, covariance = model.means_, [model.covariance_ for i in range(len(model.classes_))]
        priors = model.priors_
    elif model_name == 'QDA':
        model = QDA(store_covariance=True).fit(X, Y)
        mean, covariance = model.means_, model.covariance_
        priors = model.priors_

    fig, axes = plt.subplots(2, 2)
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]


    for j in range(2):
        axes[j, 0].scatter(X[:, 0], X[:, 1], c=Y, marker='o', cmap=cmap)
        for i in range(num_classes):
            plot_ellipse(mean[i], covariance[i], cmap(norm(i)), axes[j, 0]);

    axes[0, 0].set_title('Original Data')
    C = model.predict(xy)

    axes[1, 0].set_title('Original Classification Boundary')
    axes[1, 0].contourf(xx, yy, C.reshape(xx.shape), cmap=cmap, alpha=0.5);


    idx = get_anomalies(X, mean, covariance, priors, threshold)
    for j in range(2):
        axes[j, 1].scatter(X[idx, 0], X[idx, 1], c=Y[idx], marker='x', cmap=cmap, label='Anomalies')
        axes[j, 1].scatter(X[~idx, 0], X[~idx, 1], c=Y[~idx], marker='o', cmap=cmap, label='Non-anomalies')

    axes[0, 1].set_title('Anomaly Detection')
    axes[0, 1].legend()

    idxy = get_anomalies(xy, mean, covariance, priors, threshold)
    C = 2 * C
    C[idxy] = 1
    axes[1, 1].set_title('Anomalous Classification Boundary')
    axes[1, 1].contourf(xx, yy, C.reshape(xx.shape), cmap=cmap, alpha=0.5);
    
    for row in axes:
        for ax in row:
            ax.set_xlim(xx.min() + h, xx.max() - h)
            ax.set_ylim(yy.min() + h, yy.max() - h)
    
interact(anomaly_detection, 
         dataset=['linear', '2-blobs', 'correlated',  'imbalanced', '3-blobs', '4-blobs', 'circles', 'moons', 'iris'], 
         threshold=ipywidgets.FloatSlider(
             value=.1, min=0, max=.5, step=.01, description='Threshold', continuous_update=False),
         model_name=['Naive Bayes', 'FisherLDA', 'LDA', 'QDA'], 
         noise=ipywidgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='Noise:', continuous_update=False));
